In [1]:
%run /home/jovyan/work/operations/spark_db_connection.ipynb import create_spark_session
%run /home/jovyan/work/operations/logging_set.ipynb import set_logging
%run /home/jovyan/work/model/src/model_deployment.ipynb import Classifier, ModelTraining
%run /home/jovyan/work/operations/db_operations.ipynb import DataOperations
%run /home/jovyan/work/model/src/transform_data.ipynb import TransformData
%run /home/jovyan/work/model/process/preprocess.ipynb import preprocess
%run /home/jovyan/work/model/process/visualize.ipynb import visualize
%run /home/jovyan/work/model/process/evaluate.ipynb import evaluate
%run /home/jovyan/work/model/process/ETL.ipynb import ETL_pipeline

In [2]:
import logging

In [3]:
def classification_pipeline(epochs: int, 
                            data_path: str = "/home/jovyan/work/dataset/results.csv", 
                            table_name: str = "results") -> None:
    """
    Executes a full pipeline for training and evaluating a multi-input, multi-output, 
    multi-class neural network model for classification tasks.

    This pipeline performs the following steps:
    1. **Data Extraction**: Connects to a PostgreSQL database and saves raw data.
    2. **Data Transformation**: Transforms and prepares the data for model training, including defining schemas.
    3. **Data Loading**: Loads the data to the database.
    4. **Model Preparation**: Constructs a TensorFlow neural network model with multiple inputs and outputs.
    5. **Model Training**: Trains the model using the specified number of epochs.
    6. **Model Evaluation**: Evaluates the trained model on test data and computes performance metrics.
    7. **Visualization**: Visualizes model structure, performance and prediction results.

    Args:
        epochs: Number of epochs for training neural network
        data_path: The file path to the CSV file connecting raw data to extract
        table_name: Name of the table in PostgreSQL databse, where the data is stored
    """
    try:
        logging.info("Started classification pipeline \n")
        set_logging()
        
        spark = create_spark_session()
        etl_pipeline(spark=spark, data_path=data_path, table_name=table_name)
        
        data_oper = DataOperations(spark)
        df = data_oper.load_data(table_name="cleaned_data")

        transform_oper = TransformData()
        categorical_features, numeric_features, targets = transform_oper.describe_features_types()
        number_categories = transform_oper.number_of_categories(df, targets)

        training_dataset, test_dataset, validation_dataset, merge_models, inputs = preprocess(spark=spark,
                                                                                              categorical_features=categorical_features,
                                                                                              numeric_features=numeric_features,
                                                                                              targets=targets)
        model = Classifier()
        model_deployed, metrics = model.model_deploy(merge_models=merge_models, 
                                                     inputs=inputs, 
                                                     n_classes=number_categories)
        model_training = ModelTraining()
        trained_model, history = model_training.model_train(model=model_deployed, 
                                                            training_dataset=training_dataset, 
                                                            validation_dataset=validation_dataset,
                                                            epochs=epochs)

        home_conf_matrix, away_conf_matrix = evaluate(model=trained_model,
                                                      test_data=test_dataset,
                                                      metrics=metrics,
                                                      model_type="classifier")

        visualize(model_deployed, home_conf_matrix, away_conf_matrix, history, metrics)

        logging.info("Finished classification pipeline")
    except Exception as e:
        logging.error(f"Error while executing classification pipeline: {e}")
        raise e




In [4]:
classification_pipeline(3)

Started ETL process
Successfully ingested data from: '/home/jovyan/work/dataset/results.csv'
Successfully saved table: results
Successfully loaded table: results
Successfully converted data types for table 'results'

Successfully deleted emty fields
Successfully changed date into years
Successfully filtered data
Successfully converted string features into numeric
Successfully divided data into training, validation and test datasets
Successfully standardized datasets
Successfully saved table: cleaned_data
Successfully saved table: train
Successfully saved table: val
Successfully saved table: test
Successfully finished ETL process 

Successfully loaded table: cleaned_data
Started preprocessing
Successfully loaded table: cleaned_data
Successfully loaded table: train
Successfully loaded table: val
Successfully loaded table: test


INFO:tensorflow:Enabled check-numerics callback in thread MainThread


Enabled check-numerics callback in thread MainThread
Successfully created inputs and embedding layers for model deployment
Successfully prepared dataset for model: training
Successfully prepared dataset for model: validation
Successfully prepared dataset for model: test
Successfully finished preprocessing 



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_away_team     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_city          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_country       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_home_team     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_year          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_tournament    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 1, 50)     │     16,450 │ input_away_team[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_1         │ (None, 1, 50)     │     98,250 │ input_city[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_2         │ (None, 1, 50)     │     12,250 │ input_country[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_3         │ (None, 1, 50)     │     16,450 │ input_home_team[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_4         │ (None, 1, 23)     │      1,058 │ input_year[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_5         │ (None, 1, 50)     │      7,200 │ input_tournament… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape (Reshape)   │ (None, 50)        │          0 │ embedding[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_1 (Reshape) │ (None, 50)        │          0 │ embedding_1[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_2 (Reshape) │ (None, 50)        │          0 │ embedding_2[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_3 (Reshape) │ (None, 50)        │          0 │ embedding_3[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_4 (Reshape) │ (None, 23)        │          0 │ embedding_4[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_5 (Reshape) │ (None, 50)        │          0 │ embedding_5[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_number_featu… │ (None, 1)         │          0 │ -               

...

Successfully trained the model

Started model evaluation
Data prediction:


221/221 ━━━━━━━━━━━━━━━━━━━━ 4s 17ms/step


Successfully predicted data on the model

Calculating metrics:


221/221 ━━━━━━━━━━━━━━━━━━━━ 2s 7ms/step - away_score_loss: 1.8169 - away_score_sparse_categorical_accuracy: 0.2685 - home_score_loss: 1.6614 - home_score_sparse_categorical_accuracy: 0.2650 - loss: 3.9595



Loss (sparse_categorical_crossentropy): 3.971149444580078 
'home_score' loss (sparse_categorical_crossentropy): 1.6727094650268555 
'away_score' loss (sparse_categorical_crossentropy): 1.8203133344650269 
'home_score' sparse_categorical_accuracy: 0.27085697650909424 
'away_score' sparse_categorical_accuracy: 0.26816117763519287
Successfully finished model evaluation 

Started viusalization process
Successfully saved model visualization into file: /home/jovyan/work/model/graphs/model_schema.png
Successfully saved an image to the file: /home/jovyan/work/model/graphs/home_score_conf_mat
Successfully saved an image to the file: /home/jovyan/work/model/graphs/away_score_conf_mat
Successfully saved an image to the file: /home/jovyan/work/model/graphs/sparse_categorical_crossentropy
Successfully saved an image to the file: /home/jovyan/work/model/graphs/sparse_categorical_accuracy
Successfully finished visualization process 

Finished classification pipeline
